# Testing FLOR-1.3B Customer Service finetuned

In this Notebook, we load and test the "ericrisco/flor-1-3B-customerservice" model, a fine-tuned version of FLOR-1.3B. Hosted on Hugging Face, this model is specialized in customer service queries and answers. Our aim is to verify its functionality and assess its ability to handle specific inquiries in the customer service domain.


In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.7 MB/s eta 0:00:00
     ━━━━━━━

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id  = "ericrisco/flor-1-3B-customerservice"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/844k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/503k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

In [5]:
system_message = "Below is an instruction that describes a task. You are a helpfull customer service assistant. Answer always in catalan."

In [6]:
def prompt_template(input, response):

  full_prompt = "<|startoftext|>"
  full_prompt += "\n### Instruction\n"
  full_prompt += system_message + "\n\n"
  full_prompt += "### Context\n"
  full_prompt += input + "\n\n"
  full_prompt += "### Answer\n"
  if response != "":
    full_prompt += response
    full_prompt += "\n<|endoftext|>"

  return full_prompt

def create_prompt(sample):
  input = sample["instruction"]
  response = sample["response"]

  full_prompt = prompt_template(input, response)

  return full_prompt

In [7]:
def generate_response(prompt, model, tokenizer):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=60, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [9]:
response = generate_response(prompt_template("Com puc accedir a la meva factura?", ""),
                  model,
                  tokenizer)
print(response)

Us recomanem que llegeixi aquests consells per obrir la vostra factura. Us ajudaran a fer el procés més fàcil. Accediu a la pàgina client client facturat. Si voleu consultar-la directament, seguiu els passos que s'indiquen a la pàgina. Si necessiteu ajuda adicional, poseu-vos en
